In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
IMAGE_SIZE = [224,224]
train_path = 'brain_tumor_dataset/train'
test_path = 'brain_tumor_dataset/test'

In [3]:
vgg19 = VGG19(input_shape = IMAGE_SIZE  + [3], weights = 'imagenet', include_top = False)

In [4]:
#To avoid training the exising weights in the layers
for layer in vgg19.layers:
    layer.trainable = False

In [5]:
#This is useful in getting the number of output classes
folders = glob('brain_tumor_dataset/train/*')

In [6]:
vgg19.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [7]:
folders

['brain_tumor_dataset/train\\no', 'brain_tumor_dataset/train\\yes']

In [8]:
x = Flatten()(vgg19.output)

In [9]:
prediction = Dense(len(folders), activation = 'softmax')(x)  #Adding our output layer

#Create a model object
model = Model(inputs = vgg19.input, outputs = prediction)

In [10]:
#View the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [11]:
#telling the model what cost and optimisation method to use
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [12]:
#using the Image DataGenerator to import images from the dataset

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

Using TensorFlow backend.


In [13]:
#Making sure that you provide the same target size as initialised for the image size

training_set = train_datagen.flow_from_directory('brain_tumor_dataset/train',
                                                target_size = (224,224),
                                                batch_size = 6,
                                                class_mode = 'categorical')

Found 194 images belonging to 2 classes.


In [14]:
test_set = test_datagen.flow_from_directory('brain_tumor_dataset/test',
                                           target_size = (224,224),
                                           batch_size = 6,
                                           class_mode = 'categorical')

Found 59 images belonging to 2 classes.


In [ ]:
 #Fitting our model

r = model.fit_generator(

    training_set,
    validation_data = test_set,
    epochs = 10,
    steps_per_epoch = len(training_set),
    validation_steps = len(test_set)
)

Epoch 1/10
33/33 [==============================] - 543s 16s/step - loss: 1.1375 - accuracy: 0.6443 - val_loss: 1.4654 - val_accuracy: 0.5424
Epoch 2/10
33/33 [==============================] - 496s 15s/step - loss: 0.5653 - accuracy: 0.7835 - val_loss: 0.4592 - val_accuracy: 0.7797
Epoch 3/10
12/33 [=========>....................] - ETA: 4:43 - loss: 0.3695 - accuracy: 0.8611

In [ ]:
#Plotting the Loss
plt.plot(r.history['loss'], label = 'train_loss')
plt.plot(r.history['val_loss'], label = 'val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

#Plotting the accuracy

plt.plot(r.history['accuracy'], label = 'train_acc')
plt.plot(r.history['val_accuracy'], label = 'val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
#Saving it as a h5 file

from keras.models import load_model
model.save('BrainTumor_resnet50.h5')

from keras.models import model_from_json
# serialize model to json
json_model = model.to_json()
#save the model architecture to JSON file
with open('tumor_model.json', 'w') as json_file:
    json_file.write(json_model)
#saving the weights of the model
model.save_weights('tumor_weights.h5')